In [ ]:
# notebooks/Seu_Notebook_Principal.ipynb

import sys
import os

# Adiciona a pasta 'scripts' ao PATH do Python para que possamos importar módulos dela
# Isso é crucial se o notebook está em 'notebooks' e a função em 'scripts'
project_root = os.path.dirname(os.path.abspath('')) # Volta para o nível da 'Pasta do Projeto'
scripts_path = os.path.join(project_root, 'scripts')
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

# Agora você pode importar a função
from data_loader import load_data_with_fallback

# --- Configurações para o seu DataFrame ---
# A URL do seu arquivo CSV no GitHub
my_github_url = 'https://raw.githubusercontent.com/moises-rb/Analise_Dados/refs/heads/main/Telecom_X_PrevendoChurn/dados/dados_preprocessados.csv'
# O nome do arquivo (deve ser o mesmo no GitHub e localmente)
my_file_name = 'dados_tratados_churn.csv'

# Chama a função para carregar os dados
dados = load_data_with_fallback(
    github_raw_url=my_github_url,
    file_name=my_file_name,
    local_data_dir='dados', # Confirma que a pasta local é 'dados'
    timeout_seconds=120 # Pode ajustar o timeout aqui
)

# Verifica se o DataFrame foi carregado antes de prosseguir
if dados is not None:
    print("\nDataFrame 'dados' pronto para análise!")
else:
    print("\nNão foi possível carregar o DataFrame. Verifique as mensagens de erro acima.")

INFO: Tentando ler os dados do GitHub da URL: https://raw.githubusercontent.com/moises-rb/Analise_Dados/refs/heads/main/Telecom_X_PrevendoChurn/dados/dados_preprocessados.csv
INFO: Dados lidos com sucesso do GitHub!
INFO: DataFrame carregado com sucesso! Shape: (7032, 24)



DataFrame 'dados' pronto para análise!


In [6]:
dados.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,TenureGroup,DailyCharges,tenure_group
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,One year,Yes,Mailed check,65.6,593.30,0-12 Meses,2.186667,7-12
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,0-12 Meses,1.996667,7-12
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,0-12 Meses,2.463333,0-6
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,13-24 Meses,3.266667,13-24
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,0-12 Meses,2.796667,0-6


In [2]:
from sklearn.model_selection import train_test_split

# Separar variável alvo
X = dados.drop('Churn', axis=1)
y = dados['Churn']

# Dividir em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("✔️ Divisão feita com sucesso:")
print(f"Dados de treino: {X_train.shape}")
print(f"Dados de teste: {X_test.shape}")


✔️ Divisão feita com sucesso:
Dados de treino: (5625, 23)
Dados de teste: (1407, 23)


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Função de avaliação
def avaliar_modelo(modelo, X_test, y_test, nome_modelo):
    y_pred = modelo.predict(X_test)
    y_proba = modelo.predict_proba(X_test)[:, 1]
    print(f"\n📊 Avaliação - {nome_modelo}")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(f"AUC: {roc_auc_score(y_test, y_proba):.4f}")

# 1. Regressão Logística
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
avaliar_modelo(log_model, X_test, y_test, "Regressão Logística")

# 2. Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
avaliar_modelo(rf_model, X_test, y_test, "Random Forest")

# 3. XGBoost
#xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
#xgb_model.fit(X_train, y_train)
#avaliar_modelo(xgb_model, X_test, y_test, "XGBoost")


ValueError: could not convert string to float: '2085-JVGAD'

In [4]:
import pkg_resources
try:
    pkg_resources.get_distribution('xgboost')
    print("XGBoost is installed.")
except:
    print("XGBoost is NOT installed.")

ModuleNotFoundError: No module named 'pkg_resources'